In [25]:
%reload_ext autoreload
%autoreload 2
# BASE
import os
import json
# COOL
from pyeed.core import ProteinRecord

In [26]:
current_path = os.getcwd()

input_folder_alignment = os.path.join(current_path, "data", "alignment_protein_ids_fetch_all")
output_folder_filters = os.path.join(current_path, "data", "filtered_protein_fetch_ids_all")

In [31]:
# we read in the proteins and ids from input folder
protein_dic = {}

for file in os.listdir(input_folder_alignment):
    if file.endswith(".json"):
        with open(os.path.join(input_folder_alignment, file), "r") as f:
            # read in the file as a dic
            file_data = json.load(f)
            protein_dic[file.split('.')[0]] = {'protein': ProteinRecord.from_json_string(file_data['protein']), 'alignment': file_data['alignment']}
